# MNIST

In [1]:
import torch.utils.data
import sage
import torch
import pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as dsets
from sklearn.metrics import log_loss

E:\Anaconda1\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# Load train set
train = dsets.MNIST('../data', train=True, download=True)
imgs = train.data.reshape(-1, 784) / 255.0
labels = train.targets

# Shuffle and split into train and val
inds = torch.randperm(len(train))
print(inds.shape)
imgs = imgs[inds]
labels = labels[inds]
val, Y_val = imgs[:6000], labels[:6000]
train, Y_train = imgs[6000:], labels[6000:]

# Load test set
test = dsets.MNIST('../data', train=False, download=True)
test, Y_test = test.data.reshape(-1, 784) / 255.0, test.targets

# Move test data to numpy
test_np = test.cpu().data.numpy()
Y_test_np = Y_test.cpu().data.numpy()

torch.Size([60000])


In [3]:
def train_model(train, Y_train, val, Y_val):
    # Create model
    device = torch.device('cuda', 0)
    model = nn.Sequential(
        nn.Linear(train.shape[1], 256),
        nn.ELU(),
        nn.Linear(256, 256),
        nn.ELU(),
        nn.Linear(256, 10)).to(device)
    
    # Training parameters
    lr = 1e-3
    mbsize = 64
    max_nepochs = 250
    loss_fn = nn.CrossEntropyLoss()
    lookback = 5
    verbose = False

    # Move to GPU
    train = train.to(device)
    val = val.to(device)
    # test = test.to(device)
    Y_train = Y_train.to(device)
    Y_val = Y_val.to(device)
    # Y_test = Y_test.to(device)

    # Data loader
    train_set = TensorDataset(train, Y_train)
    train_loader = DataLoader(train_set, batch_size=mbsize, shuffle=True)

    # Setup
    optimizer = optim.Adam(model.parameters(), lr=lr)
    min_criterion = np.inf
    min_epoch = 0

    # Train
    for epoch in range(max_nepochs):
        for x, y in train_loader:
            # Move to device.
            x = x.to(device=device)
            y = y.to(device=device)

            # Take gradient step.
            loss = loss_fn(model(x), y)
            loss.backward()
            optimizer.step()
            model.zero_grad()

        # Check progress.
        with torch.no_grad():
            # Calculate validation loss.
            val_loss = loss_fn(model(val), Y_val).item()
            if verbose:
                print('{}Epoch = {}{}'.format('-' * 10, epoch + 1, '-' * 10))
                print('Val loss = {:.4f}'.format(val_loss))

            # Check convergence criterion.
            if val_loss < min_criterion:
                min_criterion = val_loss
                min_epoch = epoch
                best_model = deepcopy(model)
            elif (epoch - min_epoch) == lookback:
                if verbose:
                    print('Stopping early')
                break

    # Keep best model
    model = best_model
    return model


In [4]:
sage_values = sage.load('./results/mnist_sage.pkl')

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  2.34099571e-05  2.34370188e-05  9.77293287e-05
  1.02607823e-04  9.57539305e-05 -1.93965073e-05  1.62697164e-04
  4.39411961e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

2.303490883274465
0.0029381261266255934
0.0037325874650201746


In [5]:
psave_values=np.loadtxt("C:\\Users\\hp\\Desktop\\mnist\\metric\\psave_gau.txt")
psave_values=psave_values.flatten()

In [6]:
psave_values_gau =np.loadtxt("C:\\Users\\hp\\Desktop\\mnist\\metric\\psave_gau.txt")
psave_values_p =np.loadtxt("C:\\Users\\hp\\Desktop\\mnist\\metric\\psave_pos.txt")
psave_values_b =np.loadtxt("C:\\Users\\hp\\Desktop\\mnist\\metric\\psave_bio.txt")
psave_values_e =np.loadtxt("C:\\Users\\hp\\Desktop\\mnist\\metric\\psave_exp.txt")
psave_values_n =np.loadtxt("C:\\Users\\hp\\Desktop\\noweight.txt")
gau=np.loadtxt("C:\\Users\\hp\\Desktop\\gau.txt")
psave_values_gau=psave_values_gau.flatten()
psave_values_p=psave_values_p.flatten()
psave_values_b=psave_values_b.flatten()
psave_values_n=psave_values_n.flatten()

In [7]:
with open('results/mnist mean_importance.pkl', 'rb') as f:
    mean_imp = pickle.load(f)
print(mean_imp)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.86264515e-09 -1.67638063e-08 -1.67638063e-08
 -2.42143869e-08 -3.53902578e-08 -3.91155481e-08  2.60770321e-08
 -1.62050128e-07 -1.84401870e-07 -2.19792128e-07 -5.79282641e-07
 -4.47034836e-08 -4.09781933e-08 -1.34110451e-07 -1.06170774e-07
 -5.02914190e-08  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -2.23517

In [8]:
permutation = []
for i in range(512):
    filename = 'results/mnist permutation_test {}.pkl'.format(i)
    with open(filename, 'rb') as f:
        permutation.append(pickle.load(f)['scores'])
permutation = np.array(permutation).mean(axis=0)

784


In [9]:
with open('results/mnist feature_ablation.pkl', 'rb') as f:
    ablation = pickle.load(f)

784


In [10]:
with open('results/mnist univariate.pkl', 'rb') as f:
    univariate = pickle.load(f)

784


# Prioritized

In [8]:
num_features = list(range(5, 100, 10))
device = torch.device('cuda', 0)
print(num_features)

[5, 15, 25, 35, 45, 55, 65, 75, 85, 95]


In [24]:
importance = (psave_values_gau,psave_values_p,psave_values_b,psave_values_n,gau)
names = ('PSAVE_GAU', 'PSAVE_POS', 'PSAVE_BIO', 'PSAVE_N','GAU')
mnist_results = {name: {'values': imp} for (imp, name) in zip(importance, names)}

for name in mnist_results.keys():
    values = mnist_results[name]['values']
    order = np.argsort(values)[::-1]
    loss_list = []
    acc_list = []
    for num in num_features:
        # Subsample data
        inds = order[:num]
        inds = np.array([i in inds for i in range(784)])
        train_small = train[:, inds]
        val_small = val[:, inds]
        test_small = test[:, inds]#(10000,5)
        #Train model
        model = train_model(train_small, Y_train, val_small, Y_val)
        preds = model(test_small.to(device)).softmax(dim=1).cpu().data.numpy()/8
        loss = log_loss(Y_test_np, preds)
        acc = np.mean(np.argmax(preds, axis=1) == Y_test_np)
        loss_list.append(loss)
        acc_list.append(acc)
        print('Done with {} {} (loss = {:.4f}, acc = {:.4f})'.format(name, num, loss, acc))
    
    mnist_results[name]['accuracy'] = acc_list

Done with PSAVE_GAU 5 (loss = 1.4456, acc = 0.4798)
Done with PSAVE_GAU 15 (loss = 0.6887, acc = 0.7627)
Done with PSAVE_GAU 25 (loss = 0.4072, acc = 0.8659)
Done with PSAVE_GAU 35 (loss = 0.3274, acc = 0.8949)
Done with PSAVE_GAU 45 (loss = 0.2723, acc = 0.9105)
Done with PSAVE_GAU 55 (loss = 0.2673, acc = 0.9139)
Done with PSAVE_GAU 65 (loss = 0.2406, acc = 0.9261)
Done with PSAVE_GAU 75 (loss = 0.2184, acc = 0.9303)
Done with PSAVE_GAU 85 (loss = 0.1811, acc = 0.9436)
Done with PSAVE_GAU 95 (loss = 0.1724, acc = 0.9461)
Done with PSAVE_POS 5 (loss = 1.7981, acc = 0.3504)
Done with PSAVE_POS 15 (loss = 1.0013, acc = 0.6368)
Done with PSAVE_POS 25 (loss = 0.5489, acc = 0.8120)
Done with PSAVE_POS 35 (loss = 0.3954, acc = 0.8680)
Done with PSAVE_POS 45 (loss = 0.3028, acc = 0.9044)
Done with PSAVE_POS 55 (loss = 0.2796, acc = 0.9118)
Done with PSAVE_POS 65 (loss = 0.2390, acc = 0.9244)
Done with PSAVE_POS 75 (loss = 0.2184, acc = 0.9363)
Done with PSAVE_POS 85 (loss = 0.2006, acc = 0.9

In [ ]:
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

In [ ]:
fig, axarr = plt.subplots(1, 1, figsize=(20, 12))
num_features = list(range(5, 100, 10))

colors = ('tab:red', 'tab:gray', 'tab:green', 'tab:olive', 'tab:pink', 'tab:orange')

# Selection
ax = axarr
plt.sca(ax)
for name, color in zip(names, colors):
    values = mnist_results[name]['accuracy']
    plt.plot(num_features[4:-1], values[4:-1], color=color, label=name,
             marker='o', linestyle='--',linewidth=4)

plt.ylabel('Accuracy', fontsize=18)
plt.xlabel('# Features', fontsize=18)

plt.tick_params(labelsize=20)
plt.legend(loc='best',fontsize=30)
plt.title('MNIST Important Features', fontsize=35)

plt.show()

# Feature Selection

In [ ]:
importance = (psave_values, permutation, mean_imp, ablation, univariate,sage_values.values)
names = ('PSAVE', 'Permutation Test', 'Mean Importance', 'Feature Ablation', 'Univariate','SAGE')
mnist_results = {name: {'values': imp} for (imp, name) in zip(importance, names)}
for name in mnist_results.keys():
    values = mnist_results[name]['values']
    order = np.argsort(values)[::-1]
    loss_list = []
    acc_list = []
    for num in num_features:
        # Subsample data
        inds = order[:num]
        inds = np.array([i in inds for i in range(784)])
        train_small = train[:, inds]
        val_small = val[:, inds]
        test_small = test[:, inds]#(10000,5)
        #Train model
        model = train_model(train_small, Y_train, val_small, Y_val)
        preds = model(test_small.to(device)).softmax(dim=1).cpu().data.numpy()/8
        loss = log_loss(Y_test_np, preds)
        acc = np.mean(np.argmax(preds, axis=1) == Y_test_np)
        loss_list.append(loss)
        acc_list.append(acc)
        print('Done with {} {} (loss = {:.4f}, acc = {:.4f})'.format(name, num, loss, acc))
    
    mnist_results[name]['accuracy'] = acc_list

In [ ]:
fig, axarr = plt.subplots(1, 1, figsize=(20, 12))
num_features = list(range(5, 100, 10))

colors = ('tab:red', 'tab:gray', 'tab:green', 'tab:olive', 'tab:pink', 'tab:orange')

# Selection
ax = axarr
plt.sca(ax)
for name, color in zip(names, colors):
    values = mnist_results[name]['accuracy']
    plt.plot(num_features[4:-1], values[4:-1], color=color, label=name,
             marker='o', linestyle='--',linewidth=4)

plt.ylabel('Accuracy', fontsize=18)
plt.xlabel('# Features', fontsize=18)

plt.tick_params(labelsize=20)
plt.legend(loc='best',fontsize=30)
plt.title('MNIST Important Features', fontsize=35)

plt.show()